In [3]:
%%writefile /home/melinda_yang/greatapes/makebed_rand1000.py
##Choose random 100,000 SNPs in bed file for analysis
import random
##From wc -l Intersect_filtered_cov8.bed, 58590226 lines
#randomlines=range(700123)
#random.shuffle(randomlines)
#randomlines=randomlines[:100000]
#randomlines.sort()
#while len(randomlines) < 100000:
#    randset=set([random.randint(0,58590226) for i in range(100000)])
#    randomlines=randomlines.union(randset)
#    
#randomlines=list(randomlines)[0:100000]

randomlines=range(100000)
random.shuffle(randomlines)
randomlines=randomlines[:1000]
randomlines.sort()

pD="/home/melinda_yang/greatapes/"
#bedfile=open(pD+"Intersect_filtered_cov8_chr21.bed",'r')
#newfile=open(pD+"Intersect_filtered_cov8_chr21_rand100000.bed",'w')
bedfile=open(pD+"Intersect_filtered_cov8_chr21_rand100000.bed",'r')
newfile=open(pD+"Intersect_filtered_cov8_chr21_rand1000.bed",'w')
for ind,line in enumerate(bedfile):
    if ind not in randomlines: continue
    else: newfile.write(line)
        
bedfile.close()
newfile.close()


Writing /home/melinda_yang/greatapes/makebed_rand1000.py


In [5]:
import random
x=[1,2,3]
random.shuffle(x)
print x
x.sort()
print x

[3, 1, 2]
[1, 2, 3]


In [3]:
%%writefile /home/melinda_yang/greatapes/makegeno_cov8_rand100000.py
##Make geno file to put in diff() in R
from pysam import VariantFile
import pysam
import numpy as np

pD="/home/melinda_yang/greatapes/"
mysets="greatapes"

bedfile=open(pD+"Intersect_filtered_cov8_rand100000.bed",'r')
vcfin=VariantFile(pD+mysets+".vcf.gz")
mychr="chr21"
header = list(vcfin.header.samples)
mydiff=np.zeros((len(header),len(header)))
mygenos={"1/1":'2',"0/0":'0',"1/0":'1',"0/1":'1',"./.":'2'}
genofile=open(pD+mysets+"."+mychr+".geno",'w')
genofile.write(' '.join(header)+'\n')
snpfile=open(pD+mysets+"."+mychr+".snp",'w')

for ind,line in enumerate(bedfile):
    x=line.split()
    #if x[0] != mychr: continue
    if ind%100000==0: print line
    for pos in range(int(x[1]),int(x[2])):
        for entry in vcfin.fetch(x[0],pos-1,pos):
            alleles=list(entry.alleles)
            if len(alleles) != 2: continue
            genos=[mygenos[geno.split(':')[0]] for geno in str(entry).split('\t')[9:]]
            if len(set(genos)) == 1: continue
            genofile.write(' '.join(genos)+'\n')
            snpfile.write(x[0]+'\t'+str(pos)+'\n')
    #if num == 10: break
            #print mydiff
            #if num == 10: break
    
#np.savetxt(pD+mysets+"cov8.fixedalt."+mychr,mydiff,fmt="%.1f",comments="#".join(header))     

[4, 3, 1, 2]


In [6]:
%%writefile /home/melinda_yang/greatapes/sandpi.py
##sandpi.py

##DETERMINE S and PI

from pysam import VariantFile
import pysam
import numpy as np
import random
pD="/home/melinda_yang/greatapes/"
mysets="greatapes.fixedchr21"

##CAVEAT: ./. has same meaning as 0/0, see problem __ for why
##Give heterozygotes a count of 0.5--pretend each individual is haploid

mygenos={"1/1":1,"0/0":0,"1/0":0.5,"0/1":0.5,"./.":0} 

bedfile=open(pD+"Intersect_filtered_cov8_chr21_rand1000.bed",'r')
vcfin=VariantFile(pD+mysets+".vcf.gz")
mychr="chr21"
header = list(vcfin.header.samples)
seqary=np.zeros((1,len(header))) ##EACH ROW IS A SNP, EACH COLUMN IS AN INDIVIDUAL

num=0
for ind,line in enumerate(bedfile):
    x=line.split()
    if ind%100==0: print ind
    #if ind==100: break
    for pos in range(int(x[1]),int(x[2])):
        for entry in vcfin.fetch(x[0],pos-1,pos):
            if len(entry.alleles) != 2: continue  #print "Not BIALLELIC!";  ##Check if biallelic
            ref,alt=tuple(entry.alleles)
            genos=[mygenos[geno.split(':')[0]] for geno in str(entry).split('\t')[9:]]
            genos=[random.choice([0,1]) if i == 0.5 else i for i in genos]
            genos=np.array(genos)
            if num==0: seqary=genos
            else: seqary=np.vstack((seqary,genos))
            num+=1
np.savetxt(pD+"greatapes_Spi.txt",seqary,fmt="%.1f",comments="#".join(header))
#print header

#genera=["Gorilla","Homo","Pan","Pongo"]
#myapeinds=[sorted(random.sample([ind for ind,i in enumerate(header) if myspecies in i],9)) for myspecies in genera]
genera=['beringei','gorilla','sapiens','paniscus','troglodytes','abelii','pygmaeus']
myapeinds=[[ind for ind,i in enumerate(header) if myspecies in i] for myspecies in genera]

for ind,myape in enumerate(myapeinds):
    myary=seqary[:,myape]

    S = sum([1 for i in np.sum(myary,axis=1) if i not in [0,myary.shape[1]]]) ##Find sites where all fixed ref or alt
    pairdiff=[]
    for myind1 in range(myary.shape[1]):
        for myind2 in range(myary.shape[1]):
            if myind1>=myind2: continue
            else: pairdiff.append(np.sum(np.count_nonzero(myary[:,myind1]-myary[:,myind2])))
    pi = sum(pairdiff)/float(len(pairdiff))
    
    print genera[ind], S, pi



Overwriting /home/melinda_yang/greatapes/sandpi.py


In [24]:
%%writefile /home/melinda_yang/greatapes/sandpi.py
##sandpi.py
##Answer for Class

##DETERMINE S and PI
##input number of samples and a list of genera (separated by ',') to include for the genus
##EXAMPLE python sandpi.py 5 Homo,Gorilla

import sys
import pysam
import numpy as np
import random

pD="/home/melinda_yang/greatapes/" ##Set this to whatever directory your data is in
samplecount,genera=int(sys.argv[1]),sys.argv[2].split(',') ##Read input options into variables
bedfile=open(pD+"Intersect_filtered_cov8_chr21_rand1000.bed",'r') ##BED FILE
vcfin=pysam.VariantFile(pD+"greatapes.fixedchr21.vcf.gz") ##VCF FILE
header = list(vcfin.header.samples)  ##Get list of all IDs in header

#####      MAKE GENOTYPE ARRAY FOR ALL INDIVIDUALS
seqary=np.zeros((1,len(header))) ##EACH ROW IS A SNP, EACH COLUMN IS AN INDIVIDUAL
num=0 ##Index to count number of SNPs that we wanted to keep for analysis 
for ind,line in enumerate(bedfile): ##Loop over every set of SNPs in BED file
    x=line.split()
    ##if ind%100==0: print ind  ##A way I check to see how long it's taking to go through bed file
    for pos in range(int(x[1]),int(x[2])):
        for entry in vcfin.fetch(x[0],pos-1,pos): ##Use .fetch (pysam method) to run through and grab the line for each position in the VCF
            if len(entry.alleles) != 2: continue ##Check if biallelic; continue if not
            ref,alt=tuple(entry.alleles) ##Not necessary for computation, but way to call alleles in vcf
            ##Grab genotypes for all individuals, marking them as 0 or 1 depending on genotype
            genos=[] 
            for geno in str(entry).split('\t')[9:]:
                vcfgeno=geno.split(':')[0]
                if vcfgeno in ['1/1']: genos.append(1)
                elif vcfgeno in ['0/0','./.']: genos.append(0) ##Treat ./. as fixed for reference allele
                elif vcfgeno in ['1/0','0/1']: genos.append(random.choice([0,1])) ##Randomly choose allele for heterozygotes
                else: print 'what is vcfgeno?',vcfgeno ##Just a check to make sure I don't miss anything
            genos=np.array(genos)
            if num==0: seqary=genos ##Create array of genotypes at first line with information
            else: seqary=np.vstack((seqary,genos)) ##Add new genotypes for next line to array of genotypes
            num+=1 ##Keeping track of every SNP we used in analysis
##Save all genotype information into an array using np.savetxt (use np.loadtxt to retrieve)
np.savetxt(pD+"greatapes_genos.txt",seqary,fmt="%.1f",comments="#".join(header)) 
#####

##If you do not want to make the genotype array every run, you can comment out everything
##included in the ##### above
seqary=np.loadtxt(pD+"greatapes_genos.txt") ##Loads genotype array if you ran the part in #####

##Find random set of indices (with count based on "samplecount") for each genera included.
##Add these indices to dictionary with keys as each genera
myapeinds={}
for ind,speciesind in enumerate(header):
    genusname=speciesind.split('_')[0] ##Grab the genus from the entire individual's ID
    if genusname not in myapeinds: myapeinds[genusname]=[ind] ##Initialize each dictionary entry
    else: myapeinds[genusname].append(ind) ##Add new indices to correct dictionary entry

##Get random sample of each genus
for mygenus in genera: myapeinds[mygenus]=sorted(random.sample(myapeinds[mygenus],samplecount))
print myapeinds  ##Dictionary of indices for each genus wanted      

##This for loop goes through the genotype array and picks out the columns 
##including the individuals I want to calculate S and pi
for myape in myapeinds: 
    myary=seqary[:,myapeinds[myape]] ##Get array of genotypes for individuals you want
    ##Find S: Add 1 for each site that is not fixed reference or fixed alternative
    S=0
    for i in np.sum(myary,axis=1):
        if i not in [0,myary.shape[1]]: S+=1
        
    #Calculate pi: For every pair of individuals, subtract their array of sites and find all 
    ##non-zero entries (where the two individuals are dissimilar). These are given a value of 1
    ##using np.count_nonzero. Thus, summing across the new array of differences, gives the total 
    ##number of differences between those two individuals, which are added to a list, for which 
    ##calculate the average.
    pairdiff=[]
    for myind1 in range(myary.shape[1]):
        for myind2 in range(myary.shape[1]):
            if myind1>=myind2: continue ##Don't accidentally include a pair more than once!
            else: pairdiff.append(np.sum(np.count_nonzero(myary[:,myind1]-myary[:,myind2])))
    pi = sum(pairdiff)/float(len(pairdiff)) #Get average for pi
    
    print myape, S, pi



Overwriting /home/melinda_yang/greatapes/sandpi.py
